In [ ]:
!pip install requests beautifulsoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import re
from pprint import pprint

In [2]:
with open('resources/members.txt', 'r') as f:
    members = f.read().splitlines() 

In [3]:
URL_ROOT = 'https://forum.egosoft.com/'
URL_FORUM = 'viewforum.php?f='
URL_TOPIC = 'viewtopic.php?t='
FORUM_X4 = '146'

Parse main page

In [4]:
response = requests.get(URL_ROOT + URL_FORUM + FORUM_X4)

In [5]:
content = BeautifulSoup(response.content, 'html5lib')

In [6]:
topicsRaw = content.findAll(['a'], {'class': 'topictitle'})

In [7]:
import re

In [8]:
def topicInfo(topic):
    m = re.search('.*t=(\d*).*', topic['href'])    
    return {
        'id': m.group(1),
        'title': topic.getText(),
        'posts': topic.parent.parent.parent.find('dd',{'class','posts'}).getText().split(' ', 1)[0]
    }

In [9]:
topics = [topicInfo(t) for t in topicsRaw]

In [11]:
topic = topics[0]

In [12]:
topic

{'id': '400042', 'posts': '367', 'title': 'Screenshot of the Day'}

Parse topic pages

In [13]:
URL_ROOT + URL_TOPIC + topic['id'] + '&start=90'

'https://forum.egosoft.com/viewtopic.php?t=400042&start=90'

In [14]:
tResponse = requests.get(URL_ROOT + URL_TOPIC + topic['id'] + '&start=90')

In [32]:
tContent = BeautifulSoup(tResponse.content, 'html5lib')

In [33]:
blocksToConsider = []

for n in tContent.findAll('p', {'class': 'author'}):
    name = n.find('strong').getText().strip()
    if name in members:
        print (name)
        blocksToConsider.append(n)

CBJ
X2-Illuminatus


In [34]:
blockToConsider = blocksToConsider[0]

Parse block 

In [35]:
post = blockToConsider.parent

In [36]:
postId = post['id'][12:]

In [37]:
postId

'4730224'

In [38]:
publicationDate = re.findall('» (.*)', post.find('p', {'class': 'author'}).getText().strip())[0]
print(publicationDate)

Sun, 16. Sep 18, 15:12


In [39]:
postBody = post.find('div', {'class': 'content'})

In [40]:
children = [c for c in postBody.children]

In [41]:
child = children[0]

In [84]:
child.getText().split(' wrote:')

['AkrionXxarr',
 'Gotta say, just the idea of hanging out on that upper level and watching the M-sized docks from above as ships come and go is fairly appealing.']

In [ ]:
for c in postBody.children:
    if str(c).strip() in ['', '<br/>']:
        continue
    
    if c.findAll('cite'):
        # quotation block
        print('Author: ')
    
    print(c)
    print('---------------------')

In [42]:
postBody.getText()

"Intro\n\nOK, not daily but frequent! \n\nAs is old tradition around a release of a new X game, screenshots are a nice way to share some info as to what the game contains, even though his time around there are already many videos with footage and sound that show the game. The idea is to upload a screenshot regularly up to the release of X4 Foundations. We will add the screenshot here and also mention them as replies so that people get notifications if interested.\n\nYou are free to speculate and discuss the screenshots, but don't expect concrete answers. Even there might be some, its not a given.\n\n\n\nScreenshots\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLinks for more\n\n[INDEX] X4: Foundations - Announcement Reaction Q&A\nX4 Announcement Trailer XCON 2017 (26/08/2017)\n1st X4 Livestream: Gameplay and Q&A from XCON 2017 (English version;26/08/2017)\n2nd X4 Livestream: Stations and Geography update, with Q&As (English only; 27/09/2017)\n3rd X4 Livestream (13/12/2

In [48]:
for pb in postBody.children:
    if str(pb).strip() in ['', '<br/>']:
        continue

    chunk = {
        'type': 'text',
        'text': pb,
        'images': [],
        'links': []
    }
        
    try:
        if pb.findAll('cite'):
            chunk['type'] = 'quote'

        for img in pb.findAll('img'):
            if 'smiles' not in img['src']:
                chunk['images'].append(img['src'])

        for a in pb.findAll('a', {'class': 'postlink'}):
            chunk['links'].append(a)
        
    except AttributeError:
        pass

    pprint(chunk)

{'images': [],
 'links': [],
 'text': <blockquote><div><cite>AkrionXxarr wrote:</cite>Gotta say, just the idea of hanging out on that upper level and watching the M-sized docks from above as ships come and go is fairly appealing.</div></blockquote>,
 'type': 'quote'}
{'images': [],
 'links': [],
 'text': '\n'
         "I don't think it would be giving away too much to say that I spent "
         'some time doing exactly this yesterday. It was indeed enjoyable; I '
         'particularly liked the warning lights that started flashing before '
         'an arriving ship was even visible, giving me a chance to look out '
         'for it. I was also standing there when a fleet of small ships '
         'arrived pretty much together, creating frantic activity at the '
         'smaller docking bays.',
 'type': 'text'}
{'images': [],
 'links': [],
 'text': '\n'
         'Obviously things may change between now and release, but I would '
         'expect the general principles to remain the 

If we reach the last page, the next one will produce a general error!

In [49]:
topic['id']

'400042'

In [50]:
URL_ROOT + URL_TOPIC + topic['id'] + '&start=100'

'https://forum.egosoft.com/viewtopic.php?t=400042&start=100'

In [51]:
tResponse = requests.get(URL_ROOT + URL_TOPIC + topic['id'] + '&start=1000')

In [55]:
URL_ROOT + URL_TOPIC + topic['id'] + '&start=1000'

'https://forum.egosoft.com/viewtopic.php?t=400042&start=100'

In [52]:
tContent = BeautifulSoup(tResponse.content, 'html5lib')

In [53]:
'General Error' in tContent.getText()

False

In [69]:
p = tContent.find('div', {'class': 'pagination'})
re.search('(\d+)', p.getText()).group(0)

'369'

In [74]:
range(0, int(topic['posts']), 15)

range(0, 367, 15)

In [70]:
n = 16

In [71]:
n - (n % 15)

15